# Exploratory Data Analysis for epigenomics and other -omics datasets
**Authors**: Rachael Jin, Scott Campit

## Summary
This notebook performs exploratory data analysis between histone chromatin profiles and other datasets. Some analyses that are performed include:
  * Data summarizing 
  * Distribution analyses
  * Correlation analyses
  
**UPDATES:**
  * _10/19/2020:_ Added additional visualization code and analyses for different chromatin profiling / metabolomics datasets.

## Load data analysis libraries
First, let's load some libraries that will help with exploratory data analysis. A list of dependencies are available in the `eda_requirements.txt` file.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import kendalltau, pearsonr, spearmanr
from scipy import stats

## Explore chromatin profiles
First, we'll read in several metabolomics and chromatin profiling datasets from various studies, which are saved as individual Excel workbooks for each study. A description of each study is provided below:

  * CCLE: Global Chromatin Profiles and Metabolomics data for > 800 Cancer Cell Lines
  * Khoa: Global chromatin profiles and metabolomics data from [Khao et al., 2020](https://www.cell.com/cell-stem-cell/fulltext/S1934-5909(20)30272-1?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS1934590920302721%3Fshowall%3Dtrue)
  * Yadev: Global chromatin profiles and transcriptomics data from Yadev et al., _NaN_
  * McDonnell: Global chromatin profiles and transcriptomics data from [McDonnell et al., 2016](https://pubmed.ncbi.nlm.nih.gov/27806287/)

### Load  Chromatin Profiles
First, we'll load the global chromatin profiles (which will be denoted as by the suffix `_gcp`.

**Notes: **
  * Yadav et al has the relative abundance of the chromatin profiles in the Figures. Thus, don't worry about analyzing that data. The fold change data + FDR is present.
  * McDonnell data is obtained from 13C isotope tracing, so I need to figure out a way to analyze that data.

In [3]:
# Paths to gcp datasets
ccle_path = '~/Data/Proteomics/CCLE/CCLE Global Chromatin Profiles.xlsx'
khao_path = '~/Data/Proteomics/Dou/Khoa et al 2020 Histone PTMs.xlsx'

# Load them up as pandas dataframes
ccle_gcp = pd.read_excel(io=ccle_path, sheet_name="All")
khao_gcp = pd.read_excel(io=khao_path, sheet_name="Single modifications")

Let's view how the dataframes look like.

In [5]:
ccle_gcp.head()

,Cell Line,H3K4me0,H3K4me1,H3K4me2,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,H3K9ac1K14ac0,...,H3K27ac1K36me0,H3K27ac1K36me1,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2
0,DMS53,0.116020,-0.153144,-0.348607,-1.417128,-1.281177,-0.719707,-0.208080,-0.033416,-0.967821,...,0.396178,1.261963,0.492776,-0.211349,-0.554973,-0.222912,-0.310910,-0.272655,0.271469,0.469647
1,SW1116,-0.058624,0.219592,0.110946,-0.170282,0.334630,0.497303,0.307907,-0.466686,0.062518,...,-1.198709,-1.394997,-1.123119,-1.501911,-0.180229,-0.075173,NaN,0.051018,0.099032,0.169761
2,NCIH1694,0.480909,0.298440,0.073777,0.413953,-0.479543,0.133280,0.053279,-0.220467,-0.427160,...,0.055683,-0.659294,0.114288,-1.289012,0.280396,0.117564,NaN,0.185984,0.191760,-0.437561
3,P3HR1,-0.079957,-0.617656,-0.566702,0.079932,0.373140,0.159682,0.060946,-0.181112,0.208328,...,1.098303,0.381884,0.258282,0.751323,0.031194,-0.199316,0.037929,0.003978,-0.225147,-0.061445
4,HUT78,-0.059965,-0.063483,-0.267980,0.357422,0.075651,0.047830,0.115243,-0.498239,-0.059567,...,-0.185237,0.239421,0.358072,-0.176527,-0.351188,0.037021,NaN,0.045495,-0.153684,-0.106306


In [6]:
khao_gcp.head()

,Peptide,WT_1,WT_2,WT_3,KO_1,KO_2,KO_3,WT_MU,KO_MU,WT_STD,KO_STD,WT_CV,KO_CV,Log2FC,PValue
0,H3K14ac,0.088031,0.091734,0.098634,0.063401,0.065790,0.065056,0.092800,0.064749,0.005381,0.001224,0.0579876,0.018900,-0.519262,0.000918
1,H3K27me2,0.160086,0.205423,0.176271,0.117373,0.132054,0.118049,0.180593,0.122492,0.022975,0.008288,0.127222,0.067660,-0.560057,0.014610
2,H3K27me1,0.034532,0.032298,0.030472,0.021610,0.023388,0.013402,0.032434,0.019467,0.002033,0.005327,0.0626939,0.273640,-0.736501,0.016972
3,H3K9me3,0.119020,0.127513,0.110294,0.137611,0.135198,0.137796,0.118942,0.136868,0.008610,0.001450,0.072386,0.010591,0.202526,0.023667
4,H3K9me2,0.155693,0.148954,0.153751,0.169353,0.161655,0.177417,0.152799,0.169475,0.003469,0.007882,0.0227019,0.046507,0.149434,0.028460


### Visualize chromatin profiles
For the CCLE, there are several cancer cell lines and histone markers, while for the Khao et al paper, there are only two conditions with the same stem cell line. These will need different visualization methods.

#### Khao et al
I think I'll first visualize each column individually as a heat map, and also sanity check to see if they are

In [2]:
mt = md.drop(['Tissue', 'Medium','Culture'], axis=1)
print('\n\nmd after deleting column\n--------------')
print(mt)



md after deleting column
--------------
          CCL  2-aminoadipate  3-phosphoglycerate  Alpha-glycerophosphate  \
0       DMS53        6.112727            6.034198                5.896896   
1      SW1116        5.577413            5.727045                5.111468   
2    NCIH1694        5.886398            5.574881                5.541259   
3       P3HR1        5.770030            6.099229                6.233259   
4       HUT78        5.480683            5.469742                6.509397   
..        ...             ...                 ...                     ...   
917     SF268        5.977636            6.026483                6.480536   
918     SF539        5.957233            6.090834                5.323475   
919     SNB75        5.967707            5.931487                5.620542   
920     HOP92        5.962415            5.992640                6.296222   
921     MUTZ3        6.332344            5.812531                5.446330   

     4-pyridoxate  Aconitate   Ad

In [3]:
hm = pd.read_csv('GCP_proteomics_remapped.csv')
hm.head()


,Cell Line,H3K4me0,H3K4me1,H3K4me2,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,H3K9ac1K14ac0,...,H3K27ac1K36me0,H3K27ac1K36me1,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2
0,DMS53,0.116020,-0.153144,-0.348607,-1.417128,-1.281177,-0.719707,-0.208080,-0.033416,-0.967821,...,0.396178,1.261963,0.492776,-0.211349,-0.554973,-0.222912,-0.310910,-0.272655,0.271469,0.469647
1,SW1116,-0.058624,0.219592,0.110946,-0.170282,0.334630,0.497303,0.307907,-0.466686,0.062518,...,-1.198709,-1.394997,-1.123119,-1.501911,-0.180229,-0.075173,NaN,0.051018,0.099032,0.169761
2,NCIH1694,0.480909,0.298440,0.073777,0.413953,-0.479543,0.133280,0.053279,-0.220467,-0.427160,...,0.055683,-0.659294,0.114288,-1.289012,0.280396,0.117564,NaN,0.185984,0.191760,-0.437561
3,P3HR1,-0.079957,-0.617656,-0.566702,0.079932,0.373140,0.159682,0.060946,-0.181112,0.208328,...,1.098303,0.381884,0.258282,0.751323,0.031194,-0.199316,0.037929,0.003978,-0.225147,-0.061445
4,HUT78,-0.059965,-0.063483,-0.267980,0.357422,0.075651,0.047830,0.115243,-0.498239,-0.059567,...,-0.185237,0.239421,0.358072,-0.176527,-0.351188,0.037021,NaN,0.045495,-0.153684,-0.106306


## Merge datasets based on unique cancer cell line name
Next, we'll concatenate the two dataframes and match based on cancer cell lines.

In [4]:
merge_tb = mt.merge(hm,how='inner',left_on='CCL', right_on='Cell Line')
merge_tb.head()

,CCL,2-aminoadipate,3-phosphoglycerate,Alpha-glycerophosphate,4-pyridoxate,Aconitate,Adenine,Adipate,Alpha-ketoglutarate,AMP,...,H3K27ac1K36me0,H3K27ac1K36me1,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2
0,DMS53,6.112727,6.034198,5.896896,6.000532,5.513618,5.868529,5.977177,5.693074,5.923737,...,0.396178,1.261963,0.492776,-0.211349,-0.554973,-0.222912,-0.310910,-0.272655,0.271469,0.469647
1,SW1116,5.577413,5.727045,5.111468,6.073250,5.802494,5.824473,5.888821,5.768379,5.760784,...,-1.198709,-1.394997,-1.123119,-1.501911,-0.180229,-0.075173,NaN,0.051018,0.099032,0.169761
2,NCIH1694,5.886398,5.574881,5.541259,5.848375,5.665026,5.875548,5.894904,5.839640,5.742613,...,0.055683,-0.659294,0.114288,-1.289012,0.280396,0.117564,NaN,0.185984,0.191760,-0.437561
3,P3HR1,5.770030,6.099229,6.233259,5.543495,5.767759,6.155905,6.111148,5.949481,6.342703,...,1.098303,0.381884,0.258282,0.751323,0.031194,-0.199316,0.037929,0.003978,-0.225147,-0.061445
4,HUT78,5.480683,5.469742,6.509397,6.251005,5.190578,5.897085,6.148333,5.607481,5.871600,...,-0.185237,0.239421,0.358072,-0.176527,-0.351188,0.037021,NaN,0.045495,-0.153684,-0.106306


In [5]:
mt.info()
print('\n')
hm.info()
print('\n')
merge_tb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 922 entries, 0 to 921
Columns: 226 entries, CCL to C58:6 TAG
dtypes: float64(225), object(1)
memory usage: 1.6+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 43 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Cell Line         866 non-null    object 
 1   H3K4me0           865 non-null    float64
 2   H3K4me1           866 non-null    float64
 3   H3K4me2           865 non-null    float64
 4   H3K4ac1           790 non-null    float64
 5   H3K9me0K14ac0     866 non-null    float64
 6   H3K9me1K14ac0     866 non-null    float64
 7   H3K9me2K14ac0     865 non-null    float64
 8   H3K9me3K14ac0     864 non-null    float64
 9   H3K9ac1K14ac0     865 non-null    float64
 10  H3K9me0K14ac1     866 non-null    float64
 11  H3K9me1K14ac1     866 non-null    float64
 12  H3K9me2K14ac1     865 non-null    float64
 13  H3K9me3K14ac1     

## Separate dataframes 
Now that the data is matched by cell lines, we can separate the dataframes again.

In [6]:
metabolites = merge_tb.iloc[:,1:226]
metabolites.head()

,2-aminoadipate,3-phosphoglycerate,Alpha-glycerophosphate,4-pyridoxate,Aconitate,Adenine,Adipate,Alpha-ketoglutarate,AMP,Citrate,...,C56:8 TAG,C56:7 TAG,C56:6 TAG,C56:5 TAG,C56:4 TAG,C56:3 TAG,C56:2 TAG,C58:8 TAG,C58:7 TAG,C58:6 TAG
0,6.112727,6.034198,5.896896,6.000532,5.513618,5.868529,5.977177,5.693074,5.923737,5.641242,...,6.070239,6.133433,6.091089,6.257711,6.372732,6.202511,5.939576,6.309821,6.115974,5.999436
1,5.577413,5.727045,5.111468,6.073250,5.802494,5.824473,5.888821,5.768379,5.760784,5.914742,...,6.248653,6.633575,6.378052,6.341043,6.360945,6.333540,6.137271,7.065858,6.832174,6.363064
2,5.886398,5.574881,5.541259,5.848375,5.665026,5.875548,5.894904,5.839640,5.742613,5.570208,...,5.942887,5.946988,5.837980,5.913350,6.137530,5.807546,5.704149,5.881193,5.785208,5.504225
3,5.770030,6.099229,6.233259,5.543495,5.767759,6.155905,6.111148,5.949481,6.342703,6.054781,...,6.516922,6.113791,6.282113,6.248667,6.109480,6.043570,5.846802,6.429402,5.779815,6.241530
4,5.480683,5.469742,6.509397,6.251005,5.190578,5.897085,6.148333,5.607481,5.871600,5.128463,...,6.161981,6.777932,6.676390,6.695659,6.751029,6.385056,6.682612,6.757899,6.728570,6.879260


In [7]:
histone_markers = merge_tb.iloc[:,227:269]
histone_markers.head()

,H3K4me0,H3K4me1,H3K4me2,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,H3K9ac1K14ac0,H3K9me0K14ac1,...,H3K27ac1K36me0,H3K27ac1K36me1,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2
0,0.116020,-0.153144,-0.348607,-1.417128,-1.281177,-0.719707,-0.208080,-0.033416,-0.967821,-1.150058,...,0.396178,1.261963,0.492776,-0.211349,-0.554973,-0.222912,-0.310910,-0.272655,0.271469,0.469647
1,-0.058624,0.219592,0.110946,-0.170282,0.334630,0.497303,0.307907,-0.466686,0.062518,-0.517698,...,-1.198709,-1.394997,-1.123119,-1.501911,-0.180229,-0.075173,NaN,0.051018,0.099032,0.169761
2,0.480909,0.298440,0.073777,0.413953,-0.479543,0.133280,0.053279,-0.220467,-0.427160,0.215504,...,0.055683,-0.659294,0.114288,-1.289012,0.280396,0.117564,NaN,0.185984,0.191760,-0.437561
3,-0.079957,-0.617656,-0.566702,0.079932,0.373140,0.159682,0.060946,-0.181112,0.208328,0.182229,...,1.098303,0.381884,0.258282,0.751323,0.031194,-0.199316,0.037929,0.003978,-0.225147,-0.061445
4,-0.059965,-0.063483,-0.267980,0.357422,0.075651,0.047830,0.115243,-0.498239,-0.059567,0.845077,...,-0.185237,0.239421,0.358072,-0.176527,-0.351188,0.037021,NaN,0.045495,-0.153684,-0.106306


## Compute pearson correlation coefficient and pvalue
Finally, we'll compute the correlation coefficients between metabolites and histone markers and the p-value of correlation. Note that we're also computing metabolite-metabolite and histone-histone correlations. While those are interesting as well, we'll ignore those for downstream analyses.


In [8]:
correlation = merge_tb.corr(method ='pearson')
correlation.to_csv('correlation.csv')

In [9]:
corr = pd.concat([metabolites, histone_markers], axis=1, keys=['metabolites', 'histone_markers']).corr().loc['metabolites', 'histone_markers']
corr

,H3K4me0,H3K4me1,H3K4me2,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,H3K9ac1K14ac0,H3K9me0K14ac1,...,H3K27ac1K36me0,H3K27ac1K36me1,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2
2-aminoadipate,0.010580,-0.002126,-0.041193,-0.013648,0.050923,0.008836,-0.004897,-0.026113,0.115304,0.008429,...,0.005171,0.072125,0.102133,0.040594,-0.047334,-0.015090,0.038496,0.026004,0.021895,0.009092
3-phosphoglycerate,-0.001581,-0.019890,-0.071459,-0.084018,-0.017795,-0.053068,-0.064875,0.090798,0.034091,-0.036035,...,0.078900,0.093404,0.090150,0.026673,0.027275,-0.053514,-0.011799,-0.041461,-0.028518,-0.009319
Alpha-glycerophosphate,-0.002453,-0.045434,0.087097,0.124921,-0.046795,-0.090670,-0.040877,-0.059714,0.002468,0.093690,...,0.093395,0.051769,0.050861,0.108437,-0.126376,-0.050834,0.081335,-0.069509,-0.054641,-0.010631
4-pyridoxate,-0.037425,0.042511,0.057534,0.050476,-0.044778,-0.064867,0.001037,-0.042040,-0.047889,0.013349,...,-0.017766,-0.020870,-0.003874,0.007819,0.004127,-0.003100,0.010450,0.049615,0.011765,0.042426
Aconitate,-0.010916,0.062242,-0.017422,-0.047795,0.045195,0.042865,-0.034142,0.093691,0.071020,-0.041497,...,0.009039,0.048717,0.066045,0.035413,-0.009440,0.005705,-0.041541,-0.042933,-0.030498,-0.005861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C56:3 TAG,-0.092032,-0.124924,-0.038686,0.015128,-0.082022,-0.068769,-0.032480,-0.085301,-0.100181,-0.033661,...,-0.045226,-0.086836,-0.124022,-0.067308,-0.006703,-0.063079,0.074371,-0.052006,-0.008740,-0.021745
C56:2 TAG,-0.082580,-0.127105,-0.011321,-0.012633,-0.064240,-0.072285,-0.058981,-0.118642,-0.077662,0.027446,...,-0.026863,-0.051468,-0.096133,-0.039034,-0.074956,-0.045601,0.065746,-0.030144,-0.025248,-0.046129
C58:8 TAG,-0.022876,-0.121581,0.024798,0.007220,-0.019974,-0.020388,0.022385,-0.080380,-0.158414,-0.002622,...,0.009838,-0.062912,-0.114602,-0.066195,0.042674,-0.015420,0.075143,0.036103,-0.052345,-0.055305
C58:7 TAG,-0.037326,-0.115441,0.029761,0.016484,-0.019208,-0.009688,0.013548,-0.065745,-0.141172,-0.010678,...,-0.000221,-0.079922,-0.124879,-0.057009,0.026002,-0.017772,0.088506,0.016925,-0.071363,-0.045839


In [10]:
corr.to_csv('corr.csv')

In [11]:
def calculate_pvalues(df,mt,hm):
    """
    :param df: A pandas dataframe containing the merged table of "CCLE metabolomics dataset" and "GCP_proteomics_remapped". Rows correspond to X, Columns correspond to Y.
    :param mt: A pandas dataframe containing just the metabolites portion of the merged table.
    :param hm: A pandas dataframe containing just the histone markers portion of the mergerd table.
    :return newpvalues: A pandas dataframe containg correlations's p-values that are less or equal to 0.05
    
    """
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    df_mt = mt.dropna()._get_numeric_data()
    df_hm = hm.dropna()._get_numeric_data()
    df_mtcols = pd.DataFrame(columns=df_mt.columns)
    df_hmcols = pd.DataFrame(columns=df_hm.columns)
    newpvalues = df_mtcols.transpose().join(df_hmcols, how='outer')
    for r in (df.columns):
        for c in (df.columns):
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1],4)
    for r in (df_mt.columns):
        for c in (df_hm.columns):
            if pvalues[r][c] <= 0.05:
                newpvalues[c][r] = pvalues[r][c]
    newpvalues.to_csv('pvalues<0.05.csv')
    return newpvalues

calculate_pvalues(merge_tb,metabolites,histone_markers)

,H3K4me0,H3K4me1,H3K4me2,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,H3K9ac1K14ac0,H3K9me0K14ac1,...,H3K27ac1K36me0,H3K27ac1K36me1,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2
2-aminoadipate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0391,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3-phosphoglycerate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0027,NaN,NaN,...,0.0465,0.0107,0.0061,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alpha-glycerophosphate,NaN,NaN,0.0012,0.0002,NaN,0.0437,NaN,NaN,NaN,0.0077,...,NaN,NaN,NaN,0.0132,0.0031,NaN,0.0293,NaN,0.0434,0.032
4-pyridoxate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aconitate,NaN,0.0135,NaN,NaN,0.0153,NaN,NaN,0.0102,0.0016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C56:3 TAG,0.0155,0.0056,NaN,NaN,0.0305,NaN,NaN,0.002,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C56:2 TAG,0.0132,0.0008,NaN,NaN,NaN,NaN,NaN,0.0015,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0232,NaN,NaN,NaN,NaN,NaN
C58:8 TAG,0.0494,0.001,NaN,NaN,NaN,NaN,NaN,NaN,0.0018,NaN,...,NaN,NaN,0.0473,NaN,NaN,NaN,0.0385,NaN,NaN,NaN
C58:7 TAG,0.017,0.0031,NaN,NaN,NaN,NaN,NaN,NaN,0.0026,NaN,...,NaN,NaN,0.0453,NaN,NaN,NaN,0.0205,NaN,NaN,NaN


In [ ]:
def generate_corrtable(df,mt,hm,corr):
    """
    :param df: A pandas dataframe containing the merged table of "CCLE metabolomics dataset" and "GCP_proteomics_remapped". Rows correspond to X, Columns correspond to Y.
    :param mt: A pandas dataframe containing just the metabolites portion of the merged table.
    :param hm: A pandas dataframe containing just the histone markers portion of the mergerd table.
    :param corr: A pandas dataframe containing previous generated correlation value between metabolites and histone markers.
    :return new_corr: A pandas dataframe containg correlation that has p-values <= 0.05
    
    """
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    df_mt = mt.dropna()._get_numeric_data()
    df_hm = hm.dropna()._get_numeric_data()
    df_mtcols = pd.DataFrame(columns=df_mt.columns)
    df_hmcols = pd.DataFrame(columns=df_hm.columns)
    newpvalues = df_mtcols.transpose().join(df_hmcols, how='outer')
    new_corr = df_mtcols.transpose().join(df_hmcols, how='outer')
    for r in (df.columns):
        for c in (df.columns):
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1],4)
    for r in (df_mt.columns):
        for c in (df_hm.columns):
            if pvalues[r][c] <= 0.05:
                newpvalues[c][r] = pvalues[r][c] 
    for r in (df_mt.columns):
        for c in (df_hm.columns):
            if np.isnan(newpvalues[c][r]) == False:
                new_corr[c][r] = corr[c][r]
    new_corr.to_csv('new_corr.csv')
    return new_corr
    
generate_corrtable(merge_tb,metabolites, histone_markers, corr)